In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from __future__ import print_function

j_st = 0
j_en = 100
ncname = 'OmA_horizon_2015_newalg_' + str(j_st) + '_' + str(j_en) + '.nc'

from numpy import *
from scipy import *
import netCDF4 as nc
import numpy as np
import scipy as sp
import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools, 
    geo_tools,
    tidetools
)
from salishsea_tools import visualisations as vis
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc

import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')
import mocsy
import river_201702 as rv
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import arrow
import gsw
import datetime as dt
import timeit
start_time = timeit.default_timer()
print(start_time)

12117254.434584927


/home/tjarniko/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def find_depth_shallowalg(dp,prof,water_depth):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #tot masked
    if prof.mask.all():
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >1:
        depth_undersat = water_depth
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar)
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                if np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

In [3]:
def find_depth(dp,prof,water_depth):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #tot masked
    if prof.mask.all():
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >1:
        depth_undersat = water_depth
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar[t_indar>max_supsat])
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                if np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

In [6]:
bath = nc.Dataset('/home/sallen/MEOPAR/grid/bathymetry_201702.nc')
bath1 = bath.variables['Bathymetry'][j,i]
print(bath1)

124.0625


In [12]:
import deep_alg_as_fxn as daf
reload(daf)

<module 'deep_alg_as_fxn' from '/data/tjarniko/MEOPAR/analysis-tereza/notebooks/carbon_dev/BASE_RUN/CLEAN/OmA/deep_alg_as_fxn.py'>

In [13]:
import deep_alg_as_fxn as daf
daf.find_sathor(0,100,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(100,200,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(200,300,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(300,400,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(400,500,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(500,600,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(600,700,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(700,800,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(800,898,'DEEPALG_',False)

import deep_alg_as_fxn as daf
daf.find_sathor(0,100,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(100,200,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(200,300,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(300,400,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(400,500,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(500,600,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(600,700,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(700,800,'SHALLOWALG_',True)

import deep_alg_as_fxn as daf
daf.find_sathor(800,898,'SHALLOWALG_',True)

using deep alg
0
0
BR depth: nan PI depth: nan
1
0
BR depth: nan PI depth: nan
2
0
BR depth: nan PI depth: nan
3
0
BR depth: nan PI depth: nan
4
0
BR depth: nan PI depth: nan
5
0
BR depth: nan PI depth: nan


KeyboardInterrupt: 